In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from lib.ndcg import *

%pylab inline
import seaborn as sns
sns.set_style('darkgrid')

#some ipython options
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)

np.random.seed(0)

Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
df_train = pd.read_csv('../input/train_users_2.csv')
df_test = pd.read_csv('../input/test_users.csv')

In [3]:
#sessions_data = pd.read_csv('../input/sessions_extend_total.csv', 
#                            header = 0, sep = ',', index_col = False)
sessions_data = pd.read_csv('./sessions_action_stat.csv', 
                            header = 0, sep = ',', index_col = False)
#sessions_data = pd.read_csv('../input/sessions_reformat.csv')
#countries = pd.read_csv('../input/countries.csv') 

In [6]:
sessions_data.head()

,id,action_detail_-unknown-,action_detail_account_notification_settings,action_detail_account_payment_methods,action_detail_account_payout_preferences,action_detail_account_privacy_settings,action_detail_account_transaction_history,action_detail_admin_templates,action_detail_airbnb_picks_wishlists,action_detail_alteration_field,action_detail_alteration_request,action_detail_apply_coupon,action_detail_apply_coupon_click,action_detail_apply_coupon_click_success,action_detail_apply_coupon_error,action_detail_at_checkpoint,action_detail_book_it,action_detail_booking,action_detail_calculate_worth,action_detail_cancellation_policies,action_detail_cancellation_policy,action_detail_cancellation_policy_click,action_detail_change_availability,action_detail_change_contact_host_dates,action_detail_change_or_alter,action_detail_change_password,action_detail_change_trip_characteristics,action_detail_click_about_host,action_detail_click_amenities,action_detail_click_reviews,action_detail_complete_booking,action_detail_confirm_email,action_detail_confirm_email_link,action_detail_contact_host,action_detail_coupon_code_click,action_detail_coupon_field_focus,action_detail_create_alteration_request,action_detail_create_listing,action_detail_create_payment_instrument,action_detail_create_phone_numbers,action_detail_create_user,action_detail_dashboard,action_detail_deactivate_user_account,action_detail_delete_listing,action_detail_delete_listing_description,action_detail_delete_payment_instrument,action_detail_delete_phone_numbers,action_detail_edit_profile,action_detail_email_wishlist,action_detail_email_wishlist_button,action_detail_forgot_password,action_detail_friends_wishlists,action_detail_guest_cancellation,action_detail_guest_itinerary,action_detail_guest_receipt,action_detail_header_userpic,action_detail_homepage,action_detail_host_guarantee,action_detail_host_home,action_detail_host_refund_guest,action_detail_host_respond,action_detail_host_respond_page,action_detail_host_standard_suspension,action_detail_instant_book,action_detail_list_your_space,action_detail_listing_descriptions,action_detail_listing_recommendations,action_detail_listing_reviews,action_detail_listing_reviews_page,action_detail_login,action_detail_login_modal,action_detail_login_page,action_detail_lookup_message_thread,action_detail_manage_listing,action_detail_message_inbox,action_detail_message_post,action_detail_message_thread,action_detail_message_to_host_change,action_detail_message_to_host_focus,action_detail_modify_reservations,action_detail_modify_users,action_detail_move_map,action_detail_notifications,action_detail_oauth_login,action_detail_oauth_response,action_detail_p1,action_detail_p3,action_detail_p4,action_detail_p4_refund_policy_terms,action_detail_p4_terms,action_detail_p5,action_detail_payment_instruments,action_detail_pending,action_detail_phone_numbers,action_detail_phone_verification_success,action_detail_photos,action_detail_place_worth,action_detail_popular_wishlists,action_detail_post_checkout_action,action_detail_previous_trips,action_detail_profile_references,action_detail_profile_reviews,action_detail_profile_verifications,action_detail_read_policy_click,action_detail_remove_dashboard_alert,action_detail_request_new_confirm_email,action_detail_request_to_book,action_detail_reservations,action_detail_respond_to_alteration_request,action_detail_send_message,action_detail_set_default_payment_instrument,action_detail_set_password,action_detail_set_password_page,action_detail_share,action_detail_signup,action_detail_signup_login_page,action_detail_signup_modal,action_detail_similar_listings,action_detail_special_offer_field,action_detail_terms_and_privacy,action_detail_toggle_archived_thread,action_detail_toggle_starred_thread,action_detail_tos_2014,action_detail_translate_listing_reviews,action_detail_translations,action_detail_trip_availability,action_detail_unavailable_dates,action_detail_update_listing,action_detail_update_listing_description,

In [7]:
df_train_sess = pd.merge(df_train, 
                   sessions_data, 
                   left_on='id', 
                   right_on = 'id', 
                   how = 'left')

df_train_sess.shape

(213451, 184)

In [8]:
df_test_sess = pd.merge(df_test, 
                   sessions_data, 
                   left_on='id', 
                   right_on = 'id', 
                   how = 'left')

df_test_sess.shape

(62096, 183)

In [9]:
oneHotCols = ['signup_method']
df_all = pd.concat([df_train, df_test], axis = 0)

In [10]:
df_all = pd.get_dummies(df_all, 
                   columns = oneHotCols,
                   prefix = oneHotCols,
                   prefix_sep = '_',
                   dummy_na = False)

In [11]:
signup_methods = ['signup_method_basic',
                  'signup_method_facebook',
                  'signup_method_google',
                  'signup_method_weibo']
df_train_sess = pd.concat([df_train_sess,df_all.iloc[:213451][signup_methods]], axis = 1)
df_test_sess = pd.concat([df_test_sess,df_all.iloc[213451:][signup_methods]], axis = 1)

In [13]:
#df_train_sess.iloc[:,17:] = df_train_sess.iloc[:,17:].fillna(df_train_sess.iloc[:,17:].mean(), inplace = True)
df_train_sess.iloc[:,15:] = df_train_sess.iloc[:,15:].fillna(0)
#df_train_sess = df_train_sess.drop('user_id', axis = 1)

#df_train_sess['age'].fillna(df_train_sess['age'].mean(), inplace = True)

In [17]:
#df_test_sess.iloc[:,17:] = df_test_sess.iloc[:,17:].fillna(df_test_sess.iloc[:,17:].mean(), inplace = True)
df_test_sess.iloc[:,15:] = df_test_sess.iloc[:,15:].fillna(0)

#df_test_sess = df_test_sess.drop('user_id', axis = 1)

#df_test_sess['age'].fillna(df_test_sess['age'].mean(), inplace = True)

In [16]:
df_train_sess.to_csv('train_sess_01.csv', index_col = False)

In [18]:
df_test_sess.to_csv('test_sess_01.csv', index_col = False)

In [ ]:
def reformatLine(df_sess):
    df_line = df_sess.groupby('user_id', as_index=False).sum()
    session_list = ['action', 'action_type', 'action_detail', 'device_type']
    for col in session_list:
        session_col = list(sessions[sessions[col].notnull()][col].unique())
        temp = dict(zip(list(df_sess[col].value_counts().keys()), 
                        list(df_sess[col].value_counts())))
        count = []
        for var in session_col:
            if var in temp:
                count.append(temp.get(var))
            else:
                count.append(np.nan)
        df_tmp = pd.DataFrame(count, index = session_col).T
        df_line = pd.concat([df_line, df_tmp], axis = 1)
    return df_line

In [ ]:
with open('sessions_extend.csv', 'a') as f:
    for user_name in session_ref['user_id'][0:10]:
        df_sess = sessions[sessions['user_id'] == user_name]
        df_line = reformatLine(df_sess)
        df_line.to_csv(f, index = False, header = False)

In [ ]:
df_line = df_sess.groupby('user_id', as_index=False).sum()

session_list = ['action', 'action_type', 'action_detail', 'device_type']
for col in session_list:
    session_col = list(sessions[sessions[col].notnull()][col].unique())
    temp = dict(zip(list(df_sess[col].value_counts().keys()), 
        list(df_sess[col].value_counts())))
    count = []
    for var in session_col:
        if var in temp:
            count.append(temp.get(var))
        else:
            count.append(np.nan)
    df_tmp = pd.DataFrame(count, index = session_col).T
    df_line = pd.concat([df_line, df_tmp], axis = 1)
df_reformat = df_line

In [ ]:
df_train['timestamp_first_active'] = pd.to_datetime(df_train['timestamp_first_active'], coerce=True)

In [ ]:
df_train['date_first_booking'] = pd.to_datetime(df_train['date_first_booking'], coerce=True)

In [ ]:
df_booked['country_destination'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 8))
plt.figure()
df_notbooked['country_destination'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 8))

In [ ]:
df_booked['date_account_created'].describe()

In [ ]:
df_booked['gender'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 8))
plt.figure()
df_notbooked['gender'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 8))

In [ ]:
df_train['language'].value_counts(normalize = True)
df_train['language'].value_counts(normalize = True).plot(kind='bar', yticks = [], figsize = (10, 8))

df_test['language'].value_counts(normalize = True)
df_test['language'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 8))

plt.figure(0)
df_train['country_destination'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 8))

plt.figure(1)
df_train['gender'].value_counts(normalize = True).plot(kind='bar', figsize = (10, 3))

In [ ]:
df_agebkt = pd.read_csv('../input/age_gender_bkts.csv')

In [ ]:
ageBucket_us = df_agebkt[df_agebkt['country_destination'] == 'US']
ageBucket_us = ageBucket_us.groupby('age_bucket').sum().drop('year', axis = 1)
ageBucket_NoNus = df_agebkt[df_agebkt['country_destination'] <> 'US']
ageBucket_NoNus = ageBucket_NoNus.groupby('age_bucket').sum().drop('year', axis = 1)
ageBucket_us

In [ ]:
dt_tranUS = df_train[df_train['country_destination'] == 'US']
dt_notUS = df_train[df_train['country_destination'] <> 'US']

#dt_tranUS['date_first_booking'] = dt_tranUS['date_first_booking'].astype("datetime64")
dt_tranUS['date_first_booking'] = pd.to_datetime(dt_tranUS['date_first_booking'], coerce=True)
dt_notUS['date_first_booking'] = pd.to_datetime(dt_notUS['date_first_booking'], coerce=True)

In [ ]:
dt_tranUS['timestamp_first_active'] = pd.to_datetime(dt_tranUS['timestamp_first_active'], coerce=True)
dt_notUS['timestamp_first_active'] = pd.to_datetime(dt_notUS['timestamp_first_active'], coerce=True)


In [ ]:
dt_tranUS['date_account_created'] = pd.to_datetime(dt_tranUS['date_account_created'], coerce=True)
dt_notUS['date_account_created'] = pd.to_datetime(dt_notUS['date_account_created'], coerce=True)


In [ ]:
dt_tranUS['diff_book_created'] = dt_tranUS['date_first_booking'] - dt_tranUS['date_account_created']
dt_notUS['diff_book_created'] = dt_notUS['date_first_booking'] - dt_notUS['date_account_created']

### US / Non-US 

In [ ]:
ax = dt_tranUS['date_first_booking'].hist(bins = 80)
dt_notUS['date_first_booking'].hist(ax = ax, bins = 80)

In [ ]:
tranUS = []
for date in dt_tranUS['date_first_booking']:
    tranUS.append(date.month)

notUS = []

for date in dt_notUS['date_first_booking']:
    notUS.append(date.month)

In [ ]:
import matplotlib.pyplot as plt
import collections
sns.set(style="whitegrid")

counter  = collections.Counter(tranUS)
countNot = collections.Counter(notUS)

book_vs_month = pd.DataFrame(zip(counter.keys(), counter.values(), 
                                 countNot.values()[1:13]), 
                                 columns = ['Month', 'tran', 'Not'])

book_vs_month['tran_ratio'] = book_vs_month['tran']/book_vs_month['tran'].sum()
book_vs_month['Not_ratio'] = book_vs_month['Not']/book_vs_month['Not'].sum()

ax = sns.factorplot(x = 'Month', y = 'tran_ratio', 
                   size=6, aspect=1.5,
                   data = book_vs_month)

sns.factorplot(x = 'Month', y = 'Not_ratio', 
                   size=6, aspect=1.5,
                   data = book_vs_month)

In [ ]:
tranUS_diff = []
for x in dt_tranUS['diff_book_created']:
    tranUS_diff.append(x.days)

In [ ]:
notUS_diff = []
for x in dt_notUS['diff_book_created']:
    try:
        notUS_diff.append(x.days)
    except:
        continue

In [ ]:
plt.hist(tranUS_diff, bins = 50, label = 'tranUS')
plt.hist(notUS_diff, bins = 50, label = 'NotUS')


### diff < 0, [0, 0]; 0 < diff <= 2; [1, 0]; diff > 2 [0, 1] 

In [ ]:
sessions = pd.read_csv('../input/sessions.csv', header=0, sep=',')

In [ ]:
sessions.action.unique()